What are all the different ways that HuBMAP data can be accessed programmatically? This survey could be the basis of user-facing documentation, or it could guide efforts to align and unify resources.

Scope includes:
- HTTP APIs without Python SDK wrappers are included.
- Config files on github included, if there's nothing better.

Out of scope:
- Code and data which are not freely available.
- Tools that are only of use to HuBMAP developers.
- Software libraries not in Python.
- Bioinformatics tools for handling particular file types 

## TSV Download

**Maintainer**: Harvard

**Description**: Simple HTTP interface to pull entity metadata. URL queries supported.

**Backing API**: Search API

**Docs URL**: https://portal.hubmapconsortium.org/apis

**Source URL**: https://github.com/hubmapconsortium/portal-ui/blob/main/context/app/routes_api.py#L34

In [2]:
import csv
import io
import requests
import urllib

query = {'assay_type': 'CODEX'}
url_base = 'https://portal.hubmapconsortium.org/metadata/v0/datasets.tsv'
url_query = urllib.parse.urlencode(query)
csv_text = requests.get(f'{url_base}?{url_query}').text
datasets = [d for d in csv.DictReader(io.StringIO(csv_text), dialect=csv.excel_tab)]

datasets[0].keys()

odict_keys(['uuid', 'hubmap_id', 'acquisition_instrument_model', 'acquisition_instrument_vendor', 'analyte_class', 'assay_category', 'assay_type', 'donor.hubmap_id', 'execution_datetime', 'is_targeted', 'number_of_antibodies', 'number_of_channels', 'number_of_cycles', 'operator', 'operator_email', 'pi', 'pi_email', 'preparation_instrument_model', 'preparation_instrument_vendor', 'protocols_io_doi', 'reagent_prep_protocols_io_doi', 'resolution_x_unit', 'resolution_x_value', 'resolution_y_unit', 'resolution_y_value', 'resolution_z_unit', 'resolution_z_value', 'section_prep_protocols_io_doi'])

In [8]:
datasets[0]['preparation_instrument_model']

'The model number/name of the instrument used to prepare the sample for the assay'

In [7]:
{d['preparation_instrument_model'] for d in datasets[1:]}

{'prototype robot - Stanford/Nolan Lab', 'version 1 robot'}